In [ ]:
from PIL import Image
import torch
from torch.utils import data 
import numpy as np
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
import torchvision.models as models

In [ ]:
train_transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],
                                    std=[0.5,0.5,0.5]),
])

test_transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],
                                    std=[0.5,0.5,0.5]),
])

verif_transformer = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5,0.5,0.5],
                                    std=[0.5,0.5,0.5]),
])

In [ ]:
train_dataset=torchvision.datasets.ImageFolder(
  'D:/train',
    transform=train_transformer
)

verif_dataset=torchvision.datasets.ImageFolder(
  'D:/verif',
   transform=verif_transformer
) 

test_dataset=torchvision.datasets.ImageFolder(
  'D:/test',
   transform=test_transformer
)

In [ ]:
train_dataset.classes

In [ ]:
train_dataset.class_to_idx

In [ ]:
id_to_class={}
for k,v in train_dataset.class_to_idx.items():
    print(k,v)
    id_to_class[v]=k
id_to_class

In [ ]:
Batch_size=32
dl_train=torch.utils.data.DataLoader(
        train_dataset,
        batch_size=Batch_size,
        shuffle=True
)
dl_verif=torch.utils.data.DataLoader(
        verif_dataset,
        batch_size=Batch_size,
        shuffle=True
)
dl_test=torch.utils.data.DataLoader(
        test_dataset,
        batch_size=Batch_size,
)

In [ ]:
densenet121=models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
num_ftrs = densenet121.classifier.in_features
densenet121.classifier = torch.nn.Linear(num_ftrs, 2)
model=densenet121

In [ ]:
loss_fn=nn.CrossEntropyLoss() 

In [ ]:
from torch.optim import lr_scheduler
optim=torch.optim.Adam(model.parameters(),lr=0.000005)
scheduler = lr_scheduler.StepLR(optim,step_size=5,gamma = 0.5)

In [ ]:
if torch.cuda.is_available():
    model.to('cuda')
torch.cuda.is_available()

In [ ]:
def fit(epoch, model, trainloader, verifloader):
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x, y in tqdm(trainloader):
        if torch.cuda.is_available():
            x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
        
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
    verif_correct = 0
    verif_total = 0
    verif_running_loss = 0 
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(verifloader):
            if torch.cuda.is_available():
                x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            verif_correct += (y_pred == y).sum().item()
            verif_total += y.size(0)
            verif_running_loss += loss.item()
    
    epoch_verif_loss = verif_running_loss / len(verifloader.dataset)
    epoch_verif_acc = verif_correct / verif_total
       
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'verif_loss： ', round(epoch_verif_loss, 3),
          'verif_accuracy:', round(epoch_verif_acc, 3))
        
    return epoch_loss, epoch_acc, epoch_verif_loss, epoch_verif_acc

In [ ]:
epochs = 15

In [ ]:
patience = 3  
best_verif_loss = float('inf')
best_verif_acc = float('inf')
early_stopping_counter = 0

train_loss = []
train_acc = []
verif_loss = []
verif_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_verif_loss, epoch_verif_acc = fit(epoch,
                                                                   model,
                                                                   dl_train,
                                                                   dl_verif)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    verif_loss.append(epoch_verif_loss)
    verif_acc.append(epoch_verif_acc)

    if epoch_verif_loss < best_verif_loss or epoch_verif_acc > best_verif_acc:
        best_verif_loss = epoch_verif_loss
        best_verif_acc  = epoch_verif_acc
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

In [ ]:
plt.plot(range(1, len(train_loss)+1), train_loss, label='Train set', color='#0000FF')
plt.plot(range(1, len(verif_loss)+1), verif_loss, label='Validation set', color='#FF0000')
plt.title('Model loss function convergence curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
SFig1 = plt.gcf() 
SFig1.savefig(r'D:\densenet_LOSS.svg')

In [ ]:
plt.plot(range(1, epochs+1), train_acc, label='Train set', color='#0000FF')
plt.plot(range(1, epochs+1), verif_acc, label='Validation set', color='#FF0000')
plt.title('The accuracy of different deep learning nets in the training and validation set changes with epochs')
plt.xlabel('Epoch')
plt.ylabel('Accucary')
plt.legend()
Fig2 = plt.gcf() 
Fig2.savefig(r'D:\densenet_acc.svg')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in dl_train:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        outputs = model(inputs)
        
        _, preds = torch.max(outputs, 1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
print(cm) 

In [ ]:
all_preds2 = []
all_labels2 = []
all_pred_probs2 = []

with torch.no_grad():
    for inputs, labels in dl_verif:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        outputs = model(inputs) 
        
        _, preds = torch.max(outputs, 1)
        
        probs = F.softmax(outputs, dim=1)
        
        all_preds2.extend(preds.cpu().numpy())
        all_labels2.extend(labels.cpu().numpy())
        all_pred_probs2.extend(probs.cpu().numpy())

cm2 = confusion_matrix(all_labels2, all_preds2)
print(cm2) 


In [ ]:
all_preds1 = []
all_labels1 = []
all_pred_probs1 = []

with torch.no_grad():
    for inputs, labels in dl_test:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        outputs = model(inputs)
        
        _, preds = torch.max(outputs, 1)

        probs = F.softmax(outputs, dim=1)
        
        all_preds1.extend(preds.cpu().numpy())
        all_labels1.extend(labels.cpu().numpy())
        all_pred_probs1.extend(probs.cpu().numpy())

cm1 = confusion_matrix(all_labels1, all_preds1)
print(cm1) 


In [ ]:
torch.save(model,'densenet.pkl')

In [ ]:
import statsmodels.api as sm
from sklearn.utils import resample

In [ ]:
# Test
TN =     # nunmber of True Negatives 
TP =     # nunmber of True Positives
FP =     # nunmber of False Positives
FN =     # nunmber of False Negatives

In [ ]:
accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)
NPV = TN / (TN + FN) if (TN + FN) > 0 else 0
precision = TP / (TP + FP) if (TP + FP) > 0 else 0

def calc_confidence_interval(successes, trials, confidence=0.95):
    ci = sm.stats.proportion_confint(successes, trials, alpha=1-confidence, method='wilson')
    return ci

accuracy_ci = calc_confidence_interval(TP + TN, TP + TN + FP + FN)
sensitivity_ci = calc_confidence_interval(TP, TP + FN)
specificity_ci = calc_confidence_interval(TN, TN + FP)
NPV_ci = calc_confidence_interval(TN, TN + FN)
precision_ci = calc_confidence_interval(TP, TP + FP)
f1_score_ci = bootstrap_f1_score(TP, TN, FP, FN)

print(f"Accuracy: {accuracy:.4f} ({accuracy_ci[0]:.4f}-{accuracy_ci[1]:.4f})")
print(f"Sensitivity: {sensitivity:.4f} ({sensitivity_ci[0]:.4f}-{sensitivity_ci[1]:.4f})")
print(f"Specificity: {specificity:.4f} ({specificity_ci[0]:.4f}-{specificity_ci[1]:.4f})")
print(f"NPV: {NPV:.4f} ({NPV_ci[0]:.4f}-{NPV_ci[1]:.4f})")
print(f"Precision: {precision:.4f} ({precision_ci[0]:.4f}-{precision_ci[1]:.4f})")
print(f"F1-score: {f1_score:.4f} ({f1_score_ci[0]:.4f}-{f1_score_ci[1]:.4f})")